In [1]:
import numpy as np
import sys

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Clip DEM and visible image for rayed craters (SLDEM2015)

In [11]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids.shp'
dem = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif"
orthoimage = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif"
clip_distance = 8.0
output_dir = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters/"
output_dir_ortho = "/home/nilscp/tmp/fresh_impact_craters/LROWAC_RayedCraters/"
identifier_dem = "LROKaguyaDEM"
identifier_orthoimage = "LROWAC"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/'

In [8]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 579/579 [14:16<00:00,  1.48s/it]


In [12]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 579/579 [22:00<00:00,  2.28s/it]


## Generate regional (2.0-3.0R) detrended DEM (SLDEM2015, Rayed craters)

In [3]:
scaling_factor = 0.5
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters/"
dem_detrended_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_detrended/"
craterID = None
overwrite = True

In [ ]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

  5%|▌         | 76/1471 [00:50<10:53,  2.13it/s]

## The detection of the rim composite, and the following update of the crater centre
----
#### Note that the algorithm will not work if:
- the centre of the crater is outside of the crater of interest
- the resolution of the dem is too low (at least 20 cells is required along the crater radius to provide reasonable results <br>
----
#### It can be that the algorithm work but gives wrong results if:
- There are large change in slopes that does not correspond to the actual change in slope associated with the rim (either very close or very far)

In [4]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids.shp'
dem_folder = '/home/nilscp/tmp/fresh_impact_craters/dem_detrended/'
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/'
suffix = '_LROKaguyaDEM_detrended.tif'

In [5]:
rim_detection.main(location_of_craters, dem_folder, shp_folder, suffix, threshold_min=1500.0, threshold_max=None , craterID = None)

  0%|          | 1/1471 [00:00<03:00,  8.12it/s]/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:1033: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename)
 77%|███████▋  | 1140/1471 [31:49<17:32,  3.18s/it] 

crater1139


100%|██████████| 1471/1471 [45:16<00:00,  1.85s/it]


## Manual check of results
It is important to manually check if the detected rims give meaningful results. If not, the polygon can be deleted, and replace by a manually generated ellipse. This should (hopefully) not happen in many cases for fresh impact structures (<5%). However, this is likely to happen more often for degraded craters (such as the rayed crater dataset, that contains both young and old craters). For completion, we are writing down the id of the crater where a problem is detected, to understand if we can improve the rule-based detection of rim in the future. 

## Update centre of craters
- Merge the centres of craters together



In [3]:
out_shapefile = "/home/nilscp/tmp/fresh_impact_craters/shapefiles/final_crater_centre/global/SLDEM2015_larger_than_1500m_crater_centre_eqc_old_latlon.shp"
shp_folder = "/home/nilscp/tmp/fresh_impact_craters/shapefiles/polygons/"

In [ ]:
rim_detection.update_crater_centres(shp_folder, out_shapefile, old_crater_centres, replace_crs=False)

## Update the DEM, and the 2.0R-3.0R and 0.9R-1.1R detrendings
- decrease clip distance to 4R to divide the size of DEMs by 2 (for SLDEM2015 and SLDEM2013)
- could be that the size of the SLDEM2013 could be decreased by moving away from float.

In [4]:
location_of_craters = "/home/nilscp/tmp/fresh_impact_craters/shapefiles/final_crater_centre/global/SLDEM2015_larger_than_1500m_crater_centre_eqc_old_latlon.shp"
dem = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif"
orthoimage = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif"
clip_distance = 8.0
output_dir = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters/"
output_dir_ortho = "/home/nilscp/tmp/fresh_impact_craters/LROWAC_RayedCraters/"
identifier_dem = "LROKaguyaDEM"
identifier_orthoimage = "LROWAC"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/final_crater_centre/'

In [5]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 852/852 [21:36<00:00,  1.52s/it]


In [6]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 852/852 [17:22<00:00,  1.22s/it]


## Update the 2.0R-3.0R and 0.9R-1.1R detrendings

In [7]:
scaling_factor = 0.5
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters/"
dem_detrended_folder = "/home/nilscp/tmp/fresh_impact_craters/dem_detrended_2R3R/"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/final_crater_centre/'
craterID = None
overwrite = True

In [8]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 852/852 [05:51<00:00,  2.42it/s]


In [10]:
scaling_factor = 1.0 # first scaling factor is already applied
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/dem_detrended_2R3R/"
dem_detrended_folder = "/home/nilscp/tmp/fresh_impact_craters/dem_double_detrended/" # double detrended with 2.0R-3.0R, need to fix the part where I can specify the disk width of detrending.
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/final_crater_centre/'
craterID = None
overwrite = True

In [11]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 852/852 [05:57<00:00,  2.39it/s]


## Calculate morphometrical parameters from the second ellipse, final crater centre and new detrended products
- load ellipse in world coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r (maybe need to modify with the -xc, yc

There are a few problems by using ellipse... approximating the rim of a crater with an ellipse is totally ok, but it will always result in "shooting" over or under the actual position of the rim of crater. The majority of morphometric parameters (e.g., middle, upper, rim flank slope) are a bit sensitive to the exact location of the rim. I think I will try to create a function that retrieve the breaks in slope within 10% of the ellipse, and then use those few locations to extract cross sections and calculate parameters. This means that not all of the craters will have a constant number of cross-sections (512) as for ellipses. I will compare the results of those different values.

In [ ]:
location_of_craters = "/home/nilscp/tmp/fresh_impact_craters/shapefiles/final_crater_centre/global/SLDEM2015_larger_than_1500m_crater_centre_eqc_old_latlon.shp"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/polygons'
dem_folder = '/home/nilscp/tmp/fresh_impact_craters/dem_detrended_2R3R'
suffix = '_LROKaguyaDEM_detrended.tif'
prefix = 'crater'
craterID=None

## Small (500-1500)

In [ ]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_between-500-1500.shp'
dem = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 8.0
output_dir = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_small/"
output_dir_ortho = "/home/nilscp/tmp/fresh_impact_craters/TCOMAP_RayedCraters_small/"

identifier_dem = "LROKaguyaDEM"
identifier_orthoimage = "Kaguya_TCOMAP_02"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/'

In [ ]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

In [ ]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

In [ ]:
scaling_factor = 0.5
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_small/"
dem_detrended_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_detrended_small/"
craterID = None
overwrite = True

In [ ]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

In [ ]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids.shp'
dem_folder = '/home/nilscp/tmp/fresh_impact_craters/dem_detrended/'
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/'
suffix = '_LROKaguyaDEM_detrended.tif'

In [ ]:
rim_detection.main(location_of_craters, dem_folder, shp_folder, suffix, threshold_min=500.0, threshold_max=1500.0 , craterID = None)

In [ ]:
out_shapefile = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_between-500-1500-manually-checked.shp'
shp_folder = "/home/nilscp/tmp/fresh_impact_craters/shapefiles/polygons_small/"
old_crater_centres = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_between-500-1500.shp'

In [2]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_between-500-1500-manually-checked.shp'
dem = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif"
orthoimage = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif"
clip_distance = 8.0
output_dir = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_small/"
output_dir_ortho = "/home/nilscp/tmp/fresh_impact_craters/LROWAC_RayedCraters_small/"
identifier_dem = "LROKaguyaDEM"
identifier_orthoimage = "LROWAC"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/shapefiles/'

In [4]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 578/578 [12:41<00:00,  1.32s/it]


In [5]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 578/578 [10:40<00:00,  1.11s/it]


In [6]:
scaling_factor = 0.5
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_small/"
dem_detrended_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_detrended_small/"
craterID = None
overwrite = True

In [7]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 578/578 [03:46<00:00,  2.55it/s]


In [ ]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_between-500-1500-manually-checked.shp'
shp_folder = "/home/nilscp/tmp/fresh_impact_craters/shapefiles/polygons_small/"
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters_detrended_small/"
suffix = '_LROKaguyaDEM_detrended.tif'
prefix = 'crater'
craterID=None